# OpenStreetMaps Data Wrangling with MongoDB

In [1]:
import pymongo, pprint
cairo = pymongo.MongoClient()['osm']['cairo']

## Problems in the data
### Open/Closed Ways and Areas
The OSM Wiki describes the different types of ways here: http://wiki.openstreetmap.org/wiki/Way

The tag area='yes' is supposed to differentiate Ways that describe roads, or highways from those that describe a closed area such as a building, school or gas station. According to the Wiki, "closed ways" that dont specify the highway or barrier tags are considered to be areas. The original dataset has only 92 ways marked as areas, but many more that should be classified as areas. With the below query I classified 11,010 ways as areas and added the appropriate indicator field.

In [2]:
# find all ways with the same first and last node_ref (closed ways)
id_list = [x.get('_id') for x in cairo.aggregate([{'$match': {'type' : 'way'}},
                {'$unwind' : '$node_refs'},
                {'$group' : {'_id' : '$id',
                             'first_ref' : {'$first' : '$node_refs'}, 
                             'last_ref' : {'$last' : '$node_refs'}}},
                {'$project' : {'closed':{'$eq' : ['$first_ref', '$last_ref']}}},
                {'$match' : {'closed' : True}}])]

# update them to include area='yes'
cairo.update({'type':'way',
             'id':{'$in':id_list},
             'barrier': {'$exists':0},
             'highway': {'$exists':0}},
            {'$set':{'area' : 'yes'}}, multi=True)

{u'n': 11010, u'nModified': 10956, u'ok': 1, 'updatedExisting': True}

### Underspecified address data
A <b>remarkably low proportion</b> of the nodes have an associated address.

In [3]:
# number of nodes with address field
print 'Nodes with address:', cairo.find({'type':'node','address':{'$exists':1}}).count()
print 'Nodes with street address:', cairo.find({'type':'node','address.street':{'$exists':1}}).count()

Nodes with address: 341
Nodes with street address: 169


Only around 0.1% of nodes have any form of address data, and of those, only about half have specified streets. 

In [4]:
# number of ways with address field
print 'Ways with address:', cairo.find({'type':'way','address':{'$exists':1}}).count()
print 'Ways with street address:', cairo.find({'type':'way','address.street':{'$exists':1}}).count()

Ways with address: 1327
Ways with street address: 1031


A larger proportion (about 2.9%) of ways have some address info, and most of those that do have specified streets. The values of way.address.street are presumably the street names which define the corresponding way elements. However, that information would be more appropriate on the "name" field of that element.

This is an opportunity to add "name" values to those ways that have specified streets, but no name.

The redundancy can also form a consistency check for those ways with both fields specified. However, there is only one such way element after discounting 'area' ways.

In [5]:
num_name_and_address =  cairo.find({'type':'way',
                                    'area':{'$ne':'yes'},
                                    'name':{'$exists':1},
                                    'address.street':{'$exists':1}}).count()
num_no_name_and_address =  cairo.find({'type':'way',
                                       'area':{'$ne':'yes'},
                                       'name':{'$exists':0},
                                       'address.street':{'$exists':1}}).count()
print 'Ways with both name and street address:', num_name_and_address
print 'Ways with street address but no name:', num_no_name_and_address

Ways with both name and street address: 1
Ways with street address but no name: 62


In [6]:
# use way.address.street as way.name
cairo.update({'type':'way','area':{'$ne':'yes'}, 
              'name':{'$exists':0},'address.street':{'$exists':1}}, 
             {'$set':{'$address.street':'$name'}},multi=True)

{u'n': 62, u'nModified': 62, u'ok': 1, 'updatedExisting': True}

Interestingly, "name" is a much more commonly occuring tag for both nodes and ways. This might suggest a different usage pattern for Cairo OSM (i.e. that of using nearby named nodes/ways to locate destinations rather than their explicit address.) Named elements account for around 0.84% of all nodes and 22.59% of ways.

In [7]:
print 'Nodes with a name:', cairo.find({'type':'node','name':{'$exists':1}}).count()
print 'Ways with a name:', cairo.find({'type':'way','name':{'$exists':1}}).count()

Nodes with a name: 2766
Ways with a name: 10203


This suggests that the "address.street" field of nodes which define ways (via node_refs) should take the "name" value of the ways they define.

This can be another data consistency check as well as a method of filling in the missing values.

In [8]:
# get the nodes that are part of named ways
node_list = cairo.find({'type':'way','area':{'$ne':'yes'},'name':{'$exists':1}}).distinct('node_refs')

# count incomplete nodes
num_incomplete = cairo.find({'type':'node', 
                  'address.street' : {'$exists' : 0},
                  'id' : {'$in' : node_list}}).count()
# count nodes with both a street address and way name
num_complete = cairo.find({'type':'node', 
                  'address.street' : {'$exists' : 1},
                  'id' : {'$in' : node_list}}).count()

print 'Nodes whose street address might be filled in from "parent" way:', num_incomplete
print 'Nodes whose street address can be checked for consistency against way names:', num_complete

Nodes whose street address might be filled in from "parent" way: 52377
Nodes whose street address can be checked for consistency against way names: 2


According to this, we can add a significant number of street addresses to nodes in this way. However, since some ways describe areas and simple polygons instead of streets (as mentioned above), we should take care to exclude those from this treatment. Furthermore, some nodes are part of multiple ways, so we may need to consider those separately.

As we can see, there is not much use in a consistency check given that only 5 nodes have both fields. However, this (and any other) consistency check would be made much harder by the next problem I will discuss.

In [9]:
## USE WAY NAMES IN CONSTITUENT NODE_REFS' ADDRESS ## 
# pipeline finds nodes that are part of named ways 
pipeline = [{'$match' : {'type':'way', 
                       'area' : {'$ne':'yes'}, # ignore ways that define an area
                       'name':{'$exists':1}}},
          {'$unwind' : '$node_refs'},
          {'$group' : {'_id' : '$node_refs', 
                       'ways': {'$addToSet' : '$name'}, 
                       'count' : {'$sum' : 1}}},
          {'$match' : {'count':1}} ] # makes sure to ignore nodes that appear on more than one way

# create a dict mapping node ids to way names
node_ref_to_way_name = {x.get('_id') : x.get('ways')[0] for x in cairo.aggregate(pipeline)}

incomplete_nodes = cairo.find({'type':'node',
                               'address.street':{'$exists':0},
                               'id':{'$in':node_ref_to_way_name.keys()}})

print incomplete_nodes.count(), ' nodes to be updated.'
ignored  = []
# update incomplete nodes one by one
for node in incomplete_nodes:
    new_street_address = node_ref_to_way_name[node.get('id')]
    if isinstance(new_street_address, dict): # name field might be a dict
        try:
            new_street_address = new_street_address.get('',False) or\
                             new_street_address.get('ar',False) or\
                             new_street_address['en']
        except:
            ignored.append(new_street_address)
            continue
    cairo.update_one({'_id':node['_id']},{'$set':{'address.street' : new_street_address} })
    

37908  nodes to be updated.


In [10]:
# count incomplete nodes
num_incomplete = cairo.find({'type':'node', 
                  'address.street' : {'$exists' : 0},
                  'id' : {'$in' : node_list}}).count()
print 'Nodes whose street address remains to be filled in from "parent" way:', num_incomplete

Nodes whose street address remains to be filled in from "parent" way: 14485


### Mixture of English and Arabic

The textual fields (i.e. city, street, housename) in the data are inconsistently spelled, evenly split between their true Arabic spelling and loosely corresponding English transliterations (e.g. "Giza", "Gizeh", "الجيزة").

This, I would say, is the main problem with the Cairo OSM data. Any cleaning effort would first need to standardize the lanugage (or at least the alphabet) in which the data is represented.

Since there is no "gold standard" for transliterating between Arabic and English, there seems to be no obvious way of standardizing the data. 

One possible solution I considered was to use Google Input Tools' transliteration API. This would allow me to convert all the English transliterations into Arabic letters, but not the reverse.

### Inconsistencies with Arabic street names in nodes
* One common inconsistency is that of including or excluding the street type as in (e.g. "الهرم", "شارع الهرم") in which case the street type is "شارع" (i.e. "street").
* Another common inconsistency involves the inclusion or exclusion of the leading definite article "ال" as in ("هرم", "الهرم").

These few regular inconsistencies were standardized programatically, but with such a small number of addresses, and the added complication of them being in two different languages, the effect was not considerable.

## Data Overview

### Size of Dataset

In [11]:
ls -lh  data/cairo_egypt.osm*

-rw-r--r--@ 1 khodeir  staff    68M  7 Aug 11:09 data/cairo_egypt.osm
-rw-rw-r--@ 1 khodeir  staff    80M 17 Aug 22:27 data/cairo_egypt.osm.json


In [12]:
num_docs = cairo.find().count()
print '# of documents:', num_docs

# of documents: 372746


In [13]:
num_nodes = cairo.find({'type':'node'}).count()
print '# of nodes:', num_nodes

# of nodes: 327577


In [14]:
num_ways = cairo.find({'type':'way'}).count()
print '# of ways:', num_ways

# of ways: 45160


In [15]:
# Number of unique contributers
pipeline = [
    {
        '$group': {
            '_id' : '$created.user',
            'count' : {'$sum' : 1}
        }
    },
    {
        '$sort' : {
            'count' : -1
        }
    },
    {
        '$group' : {'_id' : None, 'num_unique' : {'$sum' : 1}}
    }
           ]

result = cairo.aggregate(pipeline)
num_users = result.next().get('num_unique')
print 'There are %d unique users'%num_users

There are 531 unique users


In [16]:
n = 20
# Sum of contributions of top n users 
pipeline = [
    {
        '$group': {
            '_id' : '$created.user',
            'count' : {'$sum' : 1}
        }
    },
    {
        '$sort' : {
            'count' : -1
        }
    },
    {
        '$limit' : n
    },
    {
        '$group' : {'_id' : None, 'total' : {'$sum' : '$count'}}
    }
           ]

result = cairo.aggregate(pipeline)
sum_top_n = result.next().get('total')

print '''Top %d users (%.2f%% of users) account for %.2f%% of contributions'''\
        % (n,100.*n/num_users,100.*sum_top_n/num_docs,)

Top 20 users (3.77% of users) account for 81.63% of contributions


In [17]:
# Users with only 1 post
pipeline = [
    {
        '$group': {
            '_id' : '$created.user',
            'count' : {'$sum' : 1}
        }
    },
    {
        '$group' : {
            '_id': '$count',
            'num_users': {'$sum' : 1}
        }
    },
    {
        '$match' : {'_id' : 1}
    },
           ]

result = cairo.aggregate(pipeline)
num_users_one_post = result.next().get('num_users')

print '''There are %d users with only one contribution. This is %.2f%% of all users.'''\
                        % (num_users_one_post,100.*num_users_one_post/num_users)

There are 98 users with only one contribution. This is 18.46% of all users.


In [18]:
# Top 10 amenities
pipeline = [
    {'$match': {'amenity':{'$exists':1}}},
    {
        '$group': {
            '_id' : '$amenity',
            'count' : {'$sum' : 1}
        }
    },
    {
        '$sort' : {
            'count' : -1
        }
    },
    {
        '$limit' : 10
    }

           ]

result = cairo.aggregate(pipeline)
pprint.pprint(list(result))



[{u'_id': u'place_of_worship', u'count': 405},
 {u'_id': u'parking', u'count': 329},
 {u'_id': u'restaurant', u'count': 202},
 {u'_id': u'school', u'count': 176},
 {u'_id': u'cafe', u'count': 148},
 {u'_id': u'fuel', u'count': 142},
 {u'_id': u'hospital', u'count': 120},
 {u'_id': u'bank', u'count': 109},
 {u'_id': u'fast_food', u'count': 101},
 {u'_id': u'pharmacy', u'count': 84}]


## Conclusion

As previously mentioned, the main problem with the Cairo OSM data is that of it being in a mixture of English and Arabic. I did not address this problem in this project, but it might be possible to use a transliteration API to convert from the English representations to their Arabic alphabet counterparts. However, the data should ideally be searchable using both alphabets, so it makes sense to input and store both representations (as is currently done with the name field.) It would help greatly if Cairo OSM users were to share a set of guidelines for formatting their data.

As part of the data cleaning I did, roughly 38,000 nodes were updated with a street address value from named ways of which they were part. I deliberately ignored nodes that appeared in more than one way, such as those that would mark the intersection of two or more roads. This leaves around  14,500 nodes that can be updated, and can be the task of further data cleaning.

Finally, it was very interesting to see that names were more common than address tags in the Cairo OSM dataset. I would also be interested in seeing how users access OSM data in Cairo as opposed to other cities. 
